# Elastodynamics with Newmark time-stepping

$u$ ... displacement, $v = \dot u $ ... velocity, $a = \dot v$ .. acceleration

Newmark scheme:

\begin{align*}
\frac{u^{n+1}-u^n}{\tau} = \frac{v^n+v^{n+1}}{2} \\
\frac{v^{n+1}-v^n}{\tau} = \frac{a^n+a^{n+1}}{2} \\
\end{align*}

with new acceleration, with elasticity operator $K$:

$$
a^{n+1} = f - K(u^{n+1})
$$

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

In [2]:
from netgen.occ import *
shape = Rectangle(1,0.1).Face()
shape.edges.Max(X).name="right"
shape.edges.Min(X).name="left"
shape.edges.Max(Y).name="top"
shape.edges.Min(Y).name="bot"
shape.vertices.Min(X+Y).maxh=0.01
shape.vertices.Min(X-Y).maxh=0.01
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [3]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def C(u):
    F = Id(2) + Grad(u)
    return F.trans * F

def NeoHooke (C):
    return 0.5*mu*(Trace(C-Id(2)) + 2*mu/lam*Det(C)**(-lam/2/mu)-1)

tau = 0.02
tend = 20
force = CF( (0, -1) )

In [4]:
fes = VectorH1(mesh, order=3, dirichlet="left")
u,v = fes.TnT()

gfu = GridFunction(fes)
gfv = GridFunction(fes)
gfa = GridFunction(fes)
gfuold = GridFunction(fes)
gfvold = GridFunction(fes)
gfaold = GridFunction(fes)

bfa = BilinearForm(fes)
bfa += Variation(NeoHooke(C(u))*dx)

vel_new = 2/tau * (u-gfuold) - gfvold
acc_new = 2/tau * (vel_new-gfvold) - gfaold

bfa += acc_new*v*dx
bfa += -force*v*dx

In [5]:
gfu_history = GridFunction(fes, multidim=0)
sceneu = Draw (gfu, deformation=True)
scenev = Draw (gfv)
gfu.vec[:] = 0
t = 0
step = 0
while t < tend:
    t += tau
    step += 1
    solvers.Newton(a=bfa, u=gfu, printing=False, inverse="sparsecholesky")

    gfv.vec[:] = 2/tau * (gfu.vec-gfuold.vec) - gfvold.vec
    gfa.vec[:] = 2/tau * (gfv.vec-gfvold.vec) - gfaold.vec

    sceneu.Redraw()
    scenev.Redraw()
    
    gfuold.vec[:] = gfu.vec
    gfvold.vec[:] = gfv.vec
    gfaold.vec[:] = gfa.vec
    if step%20 == 0: gfu_history.AddMultiDimComponent(gfu.vec)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [10]:
Draw (gfu_history, mesh, interpolate_multidim=True, animate=True, min=0, max=1, autoscale=False, deformation=True);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…